# Correcting Validity

Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).

In [10]:
import csv
import pprint

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'

def process_file(input_file, output_good, output_bad):

    bad_data = []
    good_data = []
    
    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames

        for row in reader:
            if row['productionStartYear'] == 'NULL':
#                 print('NULL')
                bad_data.append(row)
            elif row['URI'][7:18] != "dbpedia.org":
#                 print('Not dbpedia')
                continue;   
            else:
                try:
                    year = int(row['productionStartYear'][:4])
                except ValueError:
#                     print('not a proper int')
                    bad_data.append(row)
                    continue
                if year < 1886 or year > 2014:
#                     print('out of range')
                    bad_data.append(row)
                else:
#                     print("good")
                    row['productionStartYear'] = year
                    good_data.append(row)  
    
    
    with open(OUTPUT_BAD, "w") as bo:
        writer = csv.DictWriter(bo, delimiter=",", fieldnames = header)
        writer.writeheader()
        for row in bad_data:
            writer.writerow(row)
            
    with open(OUTPUT_GOOD, "w") as go:
        writer = csv.DictWriter(go, delimiter=",", fieldnames = header)
        writer.writeheader()
        for row in good_data:
            writer.writerow(row)
            



#     # This is just an example on how you can use csv.DictWriter
#     # Remember that you have to output 2 files
#     with open(output_good, "w") as g:
#         writer = csv.DictWriter(g, delimiter=",", fieldnames = header)
#         writer.writeheader()
#         for row in YOURDATA:
#             writer.writerow(row)


def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


if __name__ == "__main__":
    test()

# Profiling

#!/usr/bin/env python
#-*- coding: utf-8 -*-

In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should not be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.


In [26]:
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

INPUT_FILE = 'cities.csv'

def audit_file(filename, fields):
    fieldtypes = {}

    # My Code Here 
    with open(INPUT_FILE, "r") as f:
        reader = csv.DictReader(f)
        
        for field in FIELDS:
            fieldtypes.setdefault(field, set())
                
        for row in reader:
            if row['URI'] == 'URI' or row['URI'][:17] == "http://www.w3.org":
                continue
                
            
                
            for field in FIELDS:
                if row[field] == "NULL":
                    fieldtypes[field].add(type(None))
                elif row[field][0] == "{":
                     fieldtypes[field].add(type([]))
                else:
                    try:
                        int(row[field])
                        fieldtypes[field].add(type(1))
                    except ValueError:
                        try:
                            float(row[field])
                            fieldtypes[field].add(type(1.1))
                        except ValueError:
                            fieldtypes[field].add(type("a"))
                                    
    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)

    pprint.pprint(fieldtypes)
#     print(set([type(float()),type(list()),type(None)]) )
    assert fieldtypes["areaLand"] == set([type(float()),type(list()),type(None)]) 
    
    
if __name__ == "__main__":
    test()


{'areaCode': {<class 'list'>, <class 'NoneType'>, <class 'str'>, <class 'int'>},
 'areaLand': {<class 'float'>, <class 'NoneType'>, <class 'list'>},
 'areaMetro': {<class 'float'>, <class 'NoneType'>, <class 'list'>},
 'areaUrban': {<class 'float'>, <class 'NoneType'>, <class 'list'>},
 'elevation': {<class 'float'>, <class 'NoneType'>, <class 'list'>},
 'governmentType_label': {<class 'list'>, <class 'NoneType'>, <class 'str'>},
 'homepage': {<class 'list'>, <class 'NoneType'>, <class 'str'>},
 'isPartOf_label': {<class 'list'>, <class 'NoneType'>, <class 'str'>},
 'maximumElevation': {<class 'float'>, <class 'NoneType'>, <class 'list'>},
 'minimumElevation': {<class 'float'>, <class 'NoneType'>},
 'name': {<class 'list'>, <class 'NoneType'>, <class 'str'>},
 'populationDensity': {<class 'float'>, <class 'NoneType'>, <class 'list'>},
 'populationTotal': {<class 'list'>, <class 'NoneType'>, <class 'int'>},
 'timeZone_label': {<class 'list'>, <class 'NoneType'>, <class 'str'>},
 'utcOff

# Crossfield Auditing

In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

If you look at the full city data, you will notice that there are couple of
values that seem to provide the same information in different formats: "point"
seems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,
we do not know if that is the case and should check if they are equivalent.

Finish the function check_loc(). It will recieve 3 strings: first, the combined
value of "point" followed by the separate "wgs84_pos#" values. You have to
extract the lat and long values from the "point" argument and compare them to
the "wgs84_pos# values, returning True or False.

Note that you do not have to fix the values, only determine if they are
consistent. To fix them in this case you would need more information. Feel free
to discuss possible strategies for fixing this on the discussion forum.

Once you are done editing the code of the check_loc function, call the function process_file and examine the results.

In [36]:
import csv
import pprint

CITIES = 'cities.csv'


def check_loc(point, lat, longi):
    points = point.split(" ")
    if points[0] == lat and points[1] == longi:
        return True
    return False

def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra matadata
        for i in range(3):
            next(reader)
        # processing file
        for line in reader:
            # calling your function to check the location
            if line["point"] != "NULL" and line["point"][0] != "{": 
                result = check_loc(line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            if not result:
                print ("{}: {} != {} {}".format(line["name"], line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"]))
            data.append(line)

    return data


def test():
    assert check_loc("33.08 75.28", "33.08", "75.28") == True
    assert check_loc("44.57833333333333 -91.21833333333333", "44.5783", "-91.2183") == False

if __name__ == "__main__":
    process_file(CITIES)

Madambakkam: 12.903848 80.15861 != 12.9038 80.1586
{Palayam|Palayam Kanyakumari}: 8.483333333333333 76.91666666666667 != 8.48333 76.9167
{Pedne / पेडणें|Pernem}: 15.716944444444444 73.79777777777778 != 15.7169 73.7978
{Phulwari Sharif|फुलवारी शरीफ़}: 25.57749 85.079361 != 25.5775 85.0794
Sarupathar Bengali: 27.480555555555554 94.90916666666666 != 27.4806 94.9092
Thiruppanandal: 11.096111111111112 79.46944444444445 != 11.0961 79.4694
{Ech|Echt}: 51.1 5.883333333333333 != 51.1 5.88333
Bukuru: 9.8 8.866666666666667 != 9.8 8.86667
{Pochutla|San Pedro Pochutla}: {15.733333333333333 -96.46666666666667|15.744444444444444 -96.46527777777777} != {15.7333|15.7444} {-96.4667|-96.4653}
Bakerhill Alabama: {31.781388888888888 -85.30222222222223|31.88495 -85.1539} != {31.7814|31.885} {-85.3022|-85.1539}
{City of Santa Clarita|Santa Clarita California}: {34.416561 -118.506443|34.416666666666664 -118.50638888888889} != {34.4166|34.4167} -118.506
Murray Hill Queens: 40.76248611111111 -73.81454166666667 

City of Ontario: 34.05277777777778 -117.62777777777778 != 34.0528 -117.628
Needles California: 34.83833333333333 -114.61111111111111 != 34.8383 -114.611
City of Yucaipa: 34.030277777777776 -117.04861111111111 != 34.0303 -117.049
City of Victorville: 34.53611111111111 -117.28833333333333 != 34.5361 -117.288
{Carlsbad California|City of Carlsbad}: 33.121944444444445 -117.29694444444445 != 33.1219 -117.297
{City of Coronado|Coronado California}: 32.67805555555555 -117.1725 != 32.6781 -117.173
{Chula Vista California|City of Chula Vista}: 32.62777777777778 -117.04805555555555 != 32.6278 -117.048
{City of Del Mar|Del Mar California}: 32.955 -117.26388888888889 != 32.955 -117.264
{City of Escondido|Escondido California}: 33.124722222222225 -117.08083333333333 != 33.1247 -117.081
{City of Santee|Santee California}: 32.86972222222222 -116.97111111111111 != 32.8697 -116.971
{City of Vista|Vista California}: 33.19361111111111 -117.24111111111111 != 33.1936 -117.241
City of Escalon: 37.7916666666

Braymer Missouri: 39.589166666666664 -93.79583333333333 != 39.5892 -93.7958
Bourbon Missouri: 38.151666666666664 -91.2475 != 38.1517 -91.2475
Alma Missouri: 39.09583333333333 -93.54611111111112 != 39.0958 -93.5461
Chesterfield Missouri: 38.653333333333336 -90.55416666666666 != 38.6533 -90.5542
Clayton Missouri: 38.64527777777778 -90.33194444444445 != 38.6453 -90.3319
Sunset Hills Missouri: 38.535555555555554 -90.40388888888889 != 38.5356 -90.4039
{City of Jersey City|Jersey City New Jersey}: 40.711417 -74.06476 != 40.7114 -74.0648
Salem New Jersey: 39.568236 -75.472553 != 39.5682 -75.4725
City of Las Cruces: 32.319722222222225 -106.76527777777778 != 32.3197 -106.765
The City of ANAPRA  New Mexico: 31.806666666666665 -106.58 != 31.8067 -106.58
Lordsburg New Mexico: 32.346944444444446 -108.70722222222223 != 32.3469 -108.707
Alamogordo New Mexico: 32.9004124 -105.96047888333334 != 32.9004 -105.96
Auburn New York: 42.93333333333333 -76.56666666666666 != 42.9333 -76.5667
Jamestown: 42.09555

Naples Utah: 40.43277777777778 -109.49833333333333 != 40.4328 -109.498
American Fork Utah: 40.384166666666665 -111.79194444444444 != 40.3842 -111.792
Alpine Utah: 40.45638888888889 -111.77361111111111 != 40.4564 -111.774
Cedar Hills Utah: 40.41 -111.7575 != 40.41 -111.757
Eagle Mountain Utah: 40.306111111111115 -112.00972222222222 != 40.3061 -112.01
Elk Ridge Utah: 40.01555555555556 -111.68194444444444 != 40.0156 -111.682
Highland Utah: 40.41888888888889 -111.79222222222222 != 40.4189 -111.792
Lindon Utah: 40.33861111111111 -111.7161111111111 != 40.3386 -111.716
Lehi Utah: 40.38777777777778 -111.84916666666666 != 40.3878 -111.849
Mapleton Utah: 40.13305555555556 -111.58055555555555 != 40.1331 -111.581
Payson Utah: 40.03888888888889 -111.73305555555555 != 40.0389 -111.733
Pleasant Grove Utah: 40.37027777777778 -111.73138888888889 != 40.3703 -111.731
Salem Utah: 40.050555555555555 -111.6725 != 40.0506 -111.673
Santaquin Utah: 39.97472222222222 -111.78444444444445 != 39.9747 -111.784
Sara

{本庄|本庄市 · Honjō}: NULL != NULL NULL
{春日部|春日部市 · Kasukabe}: NULL != NULL NULL
狭山市: NULL != NULL NULL
鴻巣市: NULL != NULL NULL
深谷市: NULL != NULL NULL
上尾市: NULL != NULL NULL
{越谷|越谷市 · Koshigaya}: NULL != NULL NULL
{草加|草加市 · Sōka}: NULL != NULL NULL
蕨市: NULL != NULL NULL
入間市: NULL != NULL NULL
朝霞市: NULL != NULL NULL
新座市: NULL != NULL NULL
富士見市: NULL != NULL NULL
三郷市: NULL != NULL NULL
坂戸市: NULL != NULL NULL
幸手市: NULL != NULL NULL
{吉川市|吉川市 · Yoshikawa}: NULL != NULL NULL
{Ichikawa|市川市}: 35.71666666666667 139.93333333333334 != 35.7167 139.933
{Chōshi|銚子市}: 35.733333333333334 140.83333333333334 != 35.7333 140.833
{Tateyama|館山市}: 35.0 139.86666666666667 != 35.0 139.867
{Matsudo|松戸市}: 35.78333333333333 139.9 != 35.7833 139.9
{Noda|野田市}: 35.95 139.86666666666667 != 35.95 139.867
{Mobara|茂原市}: 35.42852777777778 140.28808333333333 != 35.4285 140.288
成田市: NULL != NULL NULL
{Tōgane|東金市}: 35.56666666666667 140.36666666666667 != 35.5667 140.367
{Asahi|旭市}: 35.71666666666667 140.65 != 35.7167 140.65
{Kas

...

South Fulton Tennessee: 36.49666666666667 -88.87916666666666 != 36.4967 -88.8792
Clarksville Tennessee: 36.52972222222222 -87.35916666666667 != 36.5297 -87.3592
Woodland Mills Tennessee: 36.47527777777778 -89.1125 != 36.4753 -89.1125
Union Tennessee: 36.42444444444445 -89.05083333333333 != 36.4244 -89.0508
Lobelville Tennessee: 35.75111111111111 -87.79305555555555 != 35.7511 -87.7931
Ducktown Tennessee: 35.034166666666664 -84.38416666666667 != 35.0342 -84.3842
Copperhill Tennessee: 34.992222222222225 -84.37416666666667 != 34.9922 -84.3742
Cookeville Tennessee: 36.16416666666667 -85.50416666666666 != 36.1642 -85.5042
Kingston Tennessee: {35.87222222222222 -84.51666666666667|35.87222222222222 -84.525} != 35.8722 {-84.525|-84.5167}
Adams Tennessee: 36.58166666666666 -87.065 != 36.5817 -87.065
Rockwood Tennessee: 35.869166666666665 -84.67527777777778 != 35.8692 -84.6753
Cedar Hill Tennessee: 36.551944444444445 -87.00222222222222 != 36.5519 -87.0022
Cross Plains Tennessee: {36.544912 -86.67

Cool Texas: {32.79833333333333 -98.01277777777777|32.798472 -98.012781} != {32.7983|32.7985} -98.0128
Hudson Oaks Texas: 32.75055555555556 -97.69694444444444 != 32.7506 -97.6969
Willow Park Texas: 32.75388888888889 -97.65 != 32.7539 -97.65
Farwell Texas: 34.38305555555556 -103.03833333333333 != 34.3831 -103.038
Bovina Texas: 34.51555555555556 -102.88805555555555 != 34.5156 -102.888
Friona Texas: 34.638888888888886 -102.72166666666666 != 34.6389 -102.722
Fort Stockton Texas: 30.89138888888889 -102.885 != 30.8914 -102.885
Iraan Texas: 30.913611111111113 -101.89861111111111 != 30.9136 -101.899
Goodrich Texas: 30.607222222222223 -94.94694444444444 != 30.6072 -94.9469
Seven Oaks Texas: 30.85361111111111 -94.85638888888889 != 30.8536 -94.8564
Onalaska Texas: 30.807777777777776 -95.10666666666667 != 30.8078 -95.1067
Presidio Texas: 29.56138888888889 -104.36638888888889 != 29.5614 -104.366
Marfa Texas: 30.311944444444446 -104.02472222222222 != 30.3119 -104.025
Point Texas: 32.93138888888889 -9

Lumberton Mississippi: 31.003888888888888 -89.45361111111112 != 31.0039 -89.4536
Baldwyn Mississippi: 34.50888888888889 -88.63944444444445 != 34.5089 -88.6394
Centralia Missouri: 39.209722222222226 -92.13638888888889 != 39.2097 -92.1364
Greentop Missouri: 40.348333333333336 -92.56722222222223 != 40.3483 -92.5672
Monett Missouri: 36.923611111111114 -93.92222222222222 != 36.9236 -93.9222
Vandalia Missouri: 39.3075 -91.48916666666666 != 39.3075 -91.4892
Drexel Missouri: 38.480555555555554 -94.60777777777778 != 38.4806 -94.6078
Dearborn Missouri: 39.5225 -94.76916666666666 != 39.5225 -94.7692
Gower Missouri: 39.61138888888889 -94.59777777777778 != 39.6114 -94.5978
Lake Ozark Missouri: 38.19861111111111 -92.63888888888889 != 38.1986 -92.6389
Richland Missouri: 37.85888888888889 -92.40277777777777 != 37.8589 -92.4028
Osage Beach Missouri: 38.138333333333335 -92.64861111111111 != 38.1383 -92.6486
Cape Girardeau Missouri: 37.30916666666667 -89.54638888888888 != 37.3092 -89.5464
Glasgow Missour

Coatzacoalcos Veracruz Llave Mexico: 18.15 -94.43333333333334 != 18.15 -94.4333
{Gyeongju|경주}: 35.85 129.21666666666667 != 35.85 129.217
{Moira Delatite Bogong|Wangaratta}: {-36.36 146.32|36.6 146.53333333333333} != {-36.36|36.6} {146.32|146.533}
{City of Dipolog|Dakbayan Sa Dipolog}: 8.583333333333334 123.33333333333333 != 8.58333 123.333
Port Augusta: {-32.483333333333334 137.76666666666668|32.483333333333334 137.76666666666668} != {-32.4833|32.4833} 137.767
Maryborough: {-25.5375 152.70194444444445|25.5375 152.70194444444445} != {-25.5375|25.5375} 152.702
{Mtskheta|მცხეთა}: 41.85 44.71666666666667 != 41.85 44.7167
Burnie: {-41.06361111111111 145.89194444444445|41.06361111111111 145.87527777777777} != {-41.0636|41.0636} {145.875|145.892}
Financial District: 40.707499 -74.011153 != 40.7075 -74.0112
Amritsar: {31.63 74.87|31.64 74.86} != {31.63|31.64} {74.86|74.87}
Barranquilla: 10.963888888888889 -74.79638888888888 != 10.9639 -74.7964
徳之島町: 27.767222222222223 129.00166666666667 != 27.

Nnewi: 6.016666666666667 6.916666666666667 != 6.01667 6.91667
La Asunción: 11.0275 -63.86277777777778 != 11.0275 -63.8628
{Brookhaven|North Atlanta Georgia}: {33.85916666666667 -84.33888888888889|33.87972222222222 -84.35166666666667} != {33.8592|33.8797} {-84.3517|-84.3389}
Cambria Heights: 40.692191666666666 -73.733025 != 40.6922 -73.733
Firozabad: {27.15 78.4|27.15 78.42} != 27.15 {78.4|78.42}
Shinyanga: -3.6619444444444444 33.42305555555556 != -3.66194 33.4231
Ramblersville: 40.65361111111111 -73.82972222222222 != 40.6536 -73.8297
Ihiala: 5.853888888888889 6.86 != 5.85389 6.86
Sutherlin Oregon: 43.388333333333335 -123.31611111111111 != 43.3883 -123.316
宇陀市: 34.516666666666666 135.96666666666667 != 34.5167 135.967
{Bien Hoa Vietnam|Thành phố Biên Hòa}: 10.95 106.81666666666666 != 10.95 106.817
Yenangyaung: 20.45826111111111 94.87318055555555 != 20.4583 94.8732
Mongu: -15.2775 23.131944444444443 != -15.2775 23.1319
Paradip: {20.16 86.4|20.32 86.62} != {20.16|20.32} {86.4|86.62}
Ocotlá